In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import tkinter as tk

In [2]:
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = os.getenv("BASE_URL")

In [4]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url,
    api_version="v2"
)

In [5]:
# List of stock tickers
symbols = ['PANW', 'LRCX', 'AAPL', 'AMZN', 'MSFT', 'TSLA', 'CAT', 'DE']

In [6]:
# Get historical data for the stock tickers
def get_historical_data(symbol, timeframe, limit):
    historical_data = api.get_bars(symbols, timeframe, limit=limit).df
    return historical_data[symbols]

In [7]:
# Simple Moving Average (SMA) calculation
def calculate_sma(data, period):
    return data.rolling(window=period).mean()

In [8]:
# Trading strategy based on SMA30 and SMA50
def trading_strategy(data):
    data['SMA30'] = calculate_sma(data['close'], 30)
    data['SMA50'] = calculate_sma(data['close'], 50)
    # Trading signals
    data['signal'] = 0
    data.loc[data['SMA30'] > data['SMA50'], 'signal'] = 1
    data.loc[data['SMA30'] < data['SMA50'], 'signal'] = -1

    return data

In [9]:
# Place trade orders
def place_order(symbol, qty, side):
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type='market',
        time_in_force='gtc'
    )

In [10]:
# Execute trades based on the selected symbol
def execute_trade():
    symbol = symbol_var.get()
    qty = int(qty_entry.get())
    side = side_var.get()# Execute trades based on the selected symbol
def execute_trade():
    symbol = symbol_var.get()
    qty = int(qty_entry.get())
    side = side_var.get()

    # Place trade order
    place_order(symbol, qty, side)

    # Clear the input fields
    symbol_var.set('')
    qty_entry.delete(0, 'end')
    side_var.set('')

In [11]:
# Create the GUI window
window = tk.Tk()
window.title('Alpaca Trading Bot')
window.geometry('300x200')

''

In [12]:
# Create and position GUI elements
symbol_label = tk.Label(window, text='Symbol:')
symbol_label.pack()
symbol_var = tk.StringVar()
symbol_entry = tk.Entry(window, textvariable=symbol_var)
symbol_entry.pack()

qty_label = tk.Label(window, text='Quantity:')
qty_label.pack()
qty_entry = tk.Entry(window)
qty_entry.pack()

side_label = tk.Label(window, text='Side:')
side_label.pack()
side_var = tk.StringVar()
side_entry = tk.Entry(window, textvariable=side_var)
side_entry.pack()

execute_button = tk.Button(window, text='Execute Trade', command=execute_trade)
execute_button.pack()

In [13]:
# Main trading loop
def main():
    for symbol in symbols:
        # Get historical data
        data = get_historical_data(symbol, '1Day', 60)

        # Apply trading strategy
        data = trading_strategy(data)

        # Check the last trading signal
        last_signal = data['signal'].iloc[-1]

        # Execute trade based on the last signal
        if last_signal == 1:
            place_order(symbol, 1, 'buy')
        elif last_signal == -1:
            place_order(symbol, 1, 'sell')

In [14]:
# Run the main trading loop
if __name__ == '__main__':
    main()

KeyError: "None of [Index(['PANW', 'LRCX', 'AAPL', 'AMZN', 'MSFT', 'TSLA', 'CAT', 'DE'], dtype='object')] are in the [columns]"

In [ ]:
# Start the GUI event loop
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\sakes\anaconda3\envs\dev\lib\site-packages\alpaca_trade_api\rest.py", line 234, in _one_request
    resp.raise_for_status()
  File "C:\Users\sakes\anaconda3\envs\dev\lib\site-packages\requests\models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 422 Client Error: Unprocessable Entity for url: https://paper-api.alpaca.markets/v2/orders

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\sakes\anaconda3\envs\dev\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\sakes\AppData\Local\Temp\ipykernel_4752\749120308.py", line 12, in execute_trade
    place_order(symbol, qty, side)
  File "C:\Users\sakes\AppData\Local\Temp\ipykernel_4752\1463840879.py", line 8, in place_order
    time_in_force='gtc'
  File "C:\Users\sakes\anaconda3